In [1]:
import requests, re, json
from tqdm import tqdm
from pocket import Pocket, PocketException
from bs4 import BeautifulSoup
from bs4.element import Comment

In [2]:
# fill these in using 
# https://getpocket.com/developer/
# and
# http://reader.fxneumann.de/plugins/oneclickpocket/auth.php
p = Pocket(
    consumer_key='',
    access_token=''
)

In [3]:
articles = p.retrieve()['list']

In [4]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    if re.match(r"[\s\r\n]+", element.encode('utf-8')): 
        return False
    return True

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def url_to_html(url):
    return requests.get(url, timeout=10).text

In [5]:
for article_id, article in tqdm(articles.items()):
    url = article['given_url']
    try:
        html = url_to_html(url)
        article['html'] = html
        article['text'] = text_from_html(html)
    except (requests.exceptions.Timeout, requests.exceptions.RequestException) as e:
        # we could not scrape the page, so it is useless
        del articles[article_id]
        print("There was an exception scraping the url '%s': %s" % (url, e))

There was an exception scraping the url 'http://techblog.netflix.com/2015/02/rad-outlier-detection-on-big-data.html': ('Connection aborted.', error(54, 'Connection reset by peer'))


There was an exception scraping the url 'http://web-rat.com/posts/2016/08/23/self-compiling-go-docker/': HTTPSConnectionPool(host='web-rat.com', port=443): Read timed out. (read timeout=10)


There was an exception scraping the url 'http://jsonfiddle.me/': HTTPConnectionPool(host='jsonfiddle.me', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11055ea90>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))


There was an exception scraping the url 'http://schier.co/blog/2015/04/26/a-simple-web-scraper-in-go.html': ('Connection aborted.', BadStatusLine("''",))


There was an exception scraping the url 'https://dataquest.io/': HTTPSConnectionPool(host='dataquest.io', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError(1, u'[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:590)'),))


100%|██████████| 379/379 [06:56<00:00,  1.10s/it]


In [6]:
with open('articles.json', 'w+') as outfile:
    json.dump(articles, outfile)